In [2]:
import torch

from scripts.MoveData import *
from scripts.Transformer import *
from scripts.TalkTrain import *

import nltk
nltk.download('wordnet', './saved/') 
from nltk.corpus import wordnet

[nltk_data] Downloading package wordnet to ./saved/...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
opt = Options(batchsize = 4)
data_iter, infield, outfield, opt = csv2datatools('saved/translation_pairs.csv','en', opt)

In [12]:
emb_dim, n_layers, heads, dropout = 64, 2, 8, 0.1 
opt.save_path = 'saved/weights/model_weights'
model = Transformer(len(infield.vocab), len(outfield.vocab), emb_dim, n_layers, heads, dropout)

if opt.device != -1:
    model = model.cuda()
    
model.load_state_dict(torch.load(opt.save_path))

<All keys matched successfully>

In [16]:
opt.lr = 0.01 # 0.01 - 0.0001
opt.epochs = 20 
optimizer = torch.optim.Adam(model.parameters(), lr=opt.lr, betas=(0.9, 0.98), eps=1e-9)
scheduler = CosineWithRestarts(optimizer, T_max=num_batches(data_iter))

In [17]:
# When the loss is less than 0.1, the responses should start to become coherent 
model = trainer(model, data_iter, opt, optimizer, scheduler)

0m: epoch 0 loss = 1.466
0m: epoch 1 loss = 0.861
0m: epoch 2 loss = 0.672
0m: epoch 3 loss = 0.532
0m: epoch 4 loss = 0.372
0m: epoch 5 loss = 0.304
0m: epoch 6 loss = 0.219
0m: epoch 7 loss = 0.151
0m: epoch 8 loss = 0.122
0m: epoch 9 loss = 0.096
0m: epoch 10 loss = 0.065
0m: epoch 11 loss = 0.053
0m: epoch 12 loss = 0.037
0m: epoch 13 loss = 0.026
0m: epoch 14 loss = 0.024
0m: epoch 15 loss = 0.015
0m: epoch 16 loss = 0.014
0m: epoch 17 loss = 0.009
0m: epoch 18 loss = 0.010
0m: epoch 19 loss = 0.008


In [18]:
sentence = "who are you?" 
opt.k = 1
opt.max_len = 10
sentence = talk_to_model(sentence, model, opt, infield, outfield)
print('Chloe > '+ sentence + '\n')

Chloe > i am chloe



In [ ]:
# when you want to turn off this cell, click Kernel->Interrupt 
while True:
    sentence = input("You > ")
    sentence = talk_to_model(sentence, model, opt, infield, outfield)
    print('Chloe > '+ sentence + '\n')

You > hi
Chloe > hello

You > who are you
Chloe > i am chloe

You > are you alive?
Chloe > depends on your definition of alive

